In [1]:
import sys, random, os, pickle
from multiprocessing import Pool
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.externals import joblib
import pandas as pd

In [2]:
if not os.path.exists('model'):
    os.mkdir('model')
if not os.path.exists('preds'):
    os.mkdir('preds')

In [3]:
train_x_d_cols = pd.read_csv('./rank_d_feature_score.csv')
train_x_d_cols = list(train_x_d_cols.iloc[10:810].feature)

In [6]:
%%time
#use rank_d  and rank_nd feature
#load data
train_x_date = pd.read_csv('../../preprocess_data/train_x_date.csv').drop(columns=['id'])
train_x_null = pd.read_csv('../../preprocess_data/train_x_null.csv').drop(columns=['id'])
train_x_int = pd.read_csv('../../preprocess_data/train_x_int.csv').drop(columns=['id','tag']) # valid中无‘tag’
train_x_d = pd.read_csv('../../preprocess_data/train_x_float_rank_d.csv',usecols=train_x_d_cols)
train_x_nd = pd.read_csv('../../preprocess_data/train_x_float_rank_nd.csv').drop(columns=['id'])

train_x = pd.concat([train_x_date,train_x_null,train_x_int,train_x_d,train_x_nd],axis=1,ignore_index=True,copy=False)
train_y = pd.read_csv('../../preprocess_data/train_y_33465.csv')
print('正负样本不均衡',train_y.label.value_counts().to_dict())

train_x = train_x.values
train_y = train_y.label.values


valid_date = pd.read_csv('../../preprocess_data/valid_date.csv')
valid_null = pd.read_csv('../../preprocess_data/valid_null.csv').drop(columns=['id'])
valid_int = pd.read_csv('../../preprocess_data/valid_int.csv').drop(columns=['id'])
valid_d = pd.read_csv('../../preprocess_data/valid_float_rank_d.csv',usecols=train_x_d_cols)
valid_nd = pd.read_csv('../../preprocess_data/valid_float_rank_nd.csv').drop(columns=['id'])
valid = pd.concat([valid_date,valid_null,valid_int,valid_d,valid_nd],axis=1,ignore_index=True,copy=False)
# pd.concat后特征名没有了，变成数字了
valid_id = valid.iloc[:,0].values
valid = valid.drop(0,axis=1)
test = valid.values
test_id = valid_id

正负样本不均衡 {0.0: 31267, 1.0: 2198}
CPU times: user 13.7 s, sys: 1.69 s, total: 15.4 s
Wall time: 15.4 s


In [7]:
def pipeline(iteration,C,gamma,random_seed):
    clf = SVC(C=C,kernel='rbf',gamma=gamma,probability=True,cache_size=8096,class_weight='balanced',verbose=True,random_state=random_seed)
    clf.fit(train_x,train_y)
    joblib.dump(clf, './model/svm{0}.pkl'.format(iteration))

    pred = clf.predict_proba(test)
    test_result = pd.DataFrame(columns=["id","score"])
    test_result.id = test_id
    test_result.score = pred[:,1]
    test_result.to_csv('./preds/svm_pred{0}.csv'.format(iteration),index=None)

In [8]:
random_seed = [x for x in range(2016,2046)]
C = [i/10.0 for i in range(10,40)]
gamma = [i/1000.0 for i in range(1,31)]
random.shuffle(random_seed)
random.shuffle(C)
random.shuffle(gamma)

with open('./params.pkl','wb') as f:
    pickle.dump((random_seed,C,gamma),f)

In [ ]:
#mp version
pool = Pool(30)
for i in range(30):
    pool.apply_async(pipeline,args=(i,C[i],gamma[i],random_seed[i]))
pool.close()
pool.join()